In [176]:
from typing import *
import libcst as cst
import networkx as nx
import pandas as pd
import numpy as np

In [177]:
from alkh import cst_utils

In [178]:
# file_path = 'play.py'
file_path = 'play2.py'
# file_path = '/mnt/dev/azure_projects/ATool/core/aoi_defects_clustering/aoi_defect_clustering.py'

In [179]:
file_lines = open(file_path, 'r').readlines()
file_content = open(file_path, 'r').read()
wrapper = cst.metadata.MetadataWrapper(cst.parse_module(file_content))
scopes = set(wrapper.resolve(cst.metadata.ScopeProvider).values())
ranges = wrapper.resolve(cst.metadata.PositionProvider)
file_number_of_lines = len(file_lines)

In [180]:
scopes

{<libcst.metadata.scope_provider.GlobalScope at 0x7f388de331f0>}

In [181]:
k = wrapper.module.body[0]
k

While(
    test=Integer(
        value='1',
        lpar=[],
        rpar=[],
    ),
    body=IndentedBlock(
        body=[
            SimpleStatementLine(
                body=[
                    Pass(
                        semicolon=MaybeSentinel.DEFAULT,
                    ),
                ],
                leading_lines=[],
                trailing_whitespace=TrailingWhitespace(
                    whitespace=SimpleWhitespace(
                        value='',
                    ),
                    comment=None,
                    newline=Newline(
                        value=None,
                    ),
                ),
            ),
        ],
        header=TrailingWhitespace(
            whitespace=SimpleWhitespace(
                value='',
            ),
            comment=None,
            newline=Newline(
                value=None,
            ),
        ),
        indent=None,
        footer=[],
    ),
    orelse=None,
    leading_lines=[],
    whitespa

In [182]:
type(k)

libcst._nodes.statement.While

In [183]:
ranges[k]

CodeRange(start=CodePosition(line=1, column=0), end=CodePosition(line=3, column=8))

In [184]:
k.test

Integer(
    value='1',
    lpar=[],
    rpar=[],
)

In [185]:
ranges[k.test]

CodeRange(start=CodePosition(line=1, column=6), end=CodePosition(line=1, column=7))

In [186]:
ranges[k.test].end.line

1

In [174]:
ranges[k]

CodeRange(start=CodePosition(line=1, column=0), end=CodePosition(line=2, column=22))

In [187]:
k

While(
    test=Integer(
        value='1',
        lpar=[],
        rpar=[],
    ),
    body=IndentedBlock(
        body=[
            SimpleStatementLine(
                body=[
                    Pass(
                        semicolon=MaybeSentinel.DEFAULT,
                    ),
                ],
                leading_lines=[],
                trailing_whitespace=TrailingWhitespace(
                    whitespace=SimpleWhitespace(
                        value='',
                    ),
                    comment=None,
                    newline=Newline(
                        value=None,
                    ),
                ),
            ),
        ],
        header=TrailingWhitespace(
            whitespace=SimpleWhitespace(
                value='',
            ),
            comment=None,
            newline=Newline(
                value=None,
            ),
        ),
        indent=None,
        footer=[],
    ),
    orelse=None,
    leading_lines=[],
    whitespa

In [189]:
class IfElseScopeCollector(cst.CSTVisitor):
    def __init__(self, ranges):
        super().__init__()
        self._ranges = ranges
        self.scopes: List[Tuple] = []

    def visit_If(self, node: cst.If) -> None:
        node_range = ranges[node]
        start_line = node_range.start.line
        head_end_line = ranges[node.whitespace_after_test].end.line
        end_line = node_range.end.line
        self.scopes.append((start_line, head_end_line, end_line))
        
    def visit_Else(self, node: cst.If) -> None:
        node_range = self._ranges[node]
        start_line = node_range.start.line
        head_end_line = self._ranges[node.whitespace_before_colon].end.line
        end_line = node_range.end.line
        self.scopes.append((start_line, head_end_line, end_line))
        
    def visit_With(self, node: cst.If) -> None:
        node_range = self._ranges[node]
        start_line = node_range.start.line
        head_end_line = self._ranges[node.whitespace_before_colon].end.line
        end_line = node_range.end.line
        self.scopes.append((start_line, head_end_line, end_line))
        
    def visit_Try(self, node: cst.If) -> None:
        node_range = self._ranges[node]
        start_line = node_range.start.line
        head_end_line = self._ranges[node.whitespace_before_colon].end.line
        end_line = node_range.end.line
        self.scopes.append((start_line, head_end_line, end_line))
        
    def visit_ExceptHandler(self, node: cst.If) -> None:
        node_range = self._ranges[node]
        start_line = node_range.start.line
        head_end_line = self._ranges[node.whitespace_before_colon].end.line
        end_line = node_range.end.line
        self.scopes.append((start_line, head_end_line, end_line))
        
    def visit_Finally(self, node: cst.If) -> None:
        node_range = self._ranges[node]
        start_line = node_range.start.line
        head_end_line = self._ranges[node.whitespace_before_colon].end.line
        end_line = node_range.end.line
        self.scopes.append((start_line, head_end_line, end_line))
        
    def visit_While(self, node: cst.While) -> None:
        node_range = self._ranges[node]
        start_line = node_range.start.line
        head_end_line = self._ranges[node.whitespace_before_colon].end.line
        end_line = node_range.end.line
        self.scopes.append((start_line, head_end_line, end_line))  

In [190]:
collector = IfElseScopeCollector(ranges)
m = wrapper.visit(collector)

In [191]:
collector.scopes

[(1, 2, 3)]

In [98]:
extended_scopes_df = pd.DataFrame(collector.scopes, columns=["start_line_number", "header_end_line_number", "end_line_number"])

,start_line_number,header_end_line_number,end_line_number
0,1,2,11
1,4,4,11
2,6,7,11
3,9,10,11


In [118]:
j = k.body.body[0]

In [119]:
ranges[j]

CodeRange(start=CodePosition(line=3, column=4), end=CodePosition(line=5, column=12))

In [120]:
j

With(
    items=[
        WithItem(
            item=Integer(
                value='1',
                lpar=[],
                rpar=[],
            ),
            asname=AsName(
                name=Name(
                    value='t',
                    lpar=[],
                    rpar=[],
                ),
                whitespace_before_as=SimpleWhitespace(
                    value=' ',
                ),
                whitespace_after_as=SimpleWhitespace(
                    value=' ',
                ),
            ),
            comma=MaybeSentinel.DEFAULT,
        ),
    ],
    body=IndentedBlock(
        body=[
            SimpleStatementLine(
                body=[
                    Pass(
                        semicolon=MaybeSentinel.DEFAULT,
                    ),
                ],
                leading_lines=[],
                trailing_whitespace=TrailingWhitespace(
                    whitespace=SimpleWhitespace(
                        value='',
        

In [122]:
ranges[j.whitespace_before_colon]

CodeRange(start=CodePosition(line=3, column=15), end=CodePosition(line=4, column=12))

In [156]:
h = k.body.body[0].body.body[0]
h

Try(
    body=IndentedBlock(
        body=[
            SimpleStatementLine(
                body=[
                    Pass(
                        semicolon=MaybeSentinel.DEFAULT,
                    ),
                ],
                leading_lines=[],
                trailing_whitespace=TrailingWhitespace(
                    whitespace=SimpleWhitespace(
                        value='',
                    ),
                    comment=None,
                    newline=Newline(
                        value=None,
                    ),
                ),
            ),
        ],
        header=TrailingWhitespace(
            whitespace=SimpleWhitespace(
                value='',
            ),
            comment=None,
            newline=Newline(
                value=None,
            ),
        ),
        indent=None,
        footer=[],
    ),
    handlers=[
        ExceptHandler(
            body=IndentedBlock(
                body=[
                    SimpleStatementLin

In [157]:
ranges[h]

CodeRange(start=CodePosition(line=5, column=8), end=CodePosition(line=11, column=16))

In [158]:
ranges[h.whitespace_before_colon]

CodeRange(start=CodePosition(line=5, column=11), end=CodePosition(line=6, column=16))